## Finite Element Solution of the 2D Heat equation
### Lars Bosch, Philipp Brückelt, Thomas Engl

----------------------------------------------------------------------------------------------------

We solve the initial boundary value problem (IBVP)
$$
\left\{ \begin{array}{rll}
\partial_{tt}^2 u - \kappa \Delta u &= f &\text{in } (0, T] \times \Omega, \\
u &= 0  &\text{on } (0, T] \times \partial \Omega),  \\
u(0, \cdot ) &= u_0 &\text{in } \Omega
\end{array} \right.
$$
where $\Omega := (0, 1)^2$, $\kappa > 0$, $u_0 ( x, y ) = \sin ( 2 \pi x ) \sin (4 \pi x)$ and $f(x) = 0$ or $f(x) = \sin ( \pi x ) \sin ( \pi y )$.

In [1]:
# import required modules

import importlib.util
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, grad, inner
import numpy as np

# for plotting / animation

import pyvista as pv
import matplotlib.pyplot as plt
from matplotlib import animation, rc, rcParams
from matplotlib.tri import Triangulation
from IPython.display import HTML
from mpi4py import MPI

# for nice output
from IPython.display import Math, display

import time

In [2]:
### copied this from the dolfinx tutorial

if importlib.util.find_spec("petsc4py") is not None:
    import dolfinx

    if not dolfinx.has_petsc:
        print("This demo requires DOLFINx to be compiled with PETSc enabled.")
        exit(0)
    from petsc4py.PETSc import ScalarType  # type: ignore
else:
    print("This demo requires petsc4py.")
    exit(0)

At the start, please chose which example you want to see. Either 0 or 1, where 0 means that the right hand side is given by
$$
f \equiv 0.
$$
If you chose 1, the right hand side is given by
$$
f(x, y) = \sin ( \pi x ) \sin ( \pi y ).
$$

In [3]:
# set example to either 0 or 1
# initial conditions and analytic solutions are defined later
example = 1

Create the mesh, function space and impose boundary conditions

In [4]:
### =======================================================================
### solve the two-dimensional heat equation in \Omega = (0, 1)^2
### u_t - \kappa \Delta u = f    in \Omega
###                     u = 0    on \partial \Omega \times (0, T)
###               u(0, -) = u_0


### =======================================================================
### create a triangular mesh on the unit square (0, 1)^2 with 128 cells 
### in each direction
### We use a high number of cells since there are large oscillations in the
### initial condition (see later)

msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0, 0.0), (1.0, 1.0)),
    n=(128, 128),
    cell_type=mesh.CellType.triangle,
)

# use Lagrange elements of degree 1
V = fem.functionspace(msh, ("Lagrange", 1))

# implement homogeneous Dirichlet boundary conditions
# u = 0 on the boundary, i.e., for x \in \{0, 1\} and y \in \{0, 1\}

facets = mesh.locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0) |
                     np.isclose(x[1], 0.0) | np.isclose(x[1], 1.0),
)

# find degrees of freedom

dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)

# create a fem class representing the boundary conditions

bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [5]:
# Iterate over all time steps: Choose h = 0.001 as step size
# this ensures that the development of the solution in time can be
# observed well in the animation

h = 0.001

x = ufl.SpatialCoordinate(msh)


Define the functions $u_0$ and $f$.

In [6]:
### =====================================================================
### initial condition u_0
### u_0 (x, y) = sin(pi x) sin(pi y) + sin(2 pi x) sin(4 pi y)
### create a ufl expression and transform it into a fem function later

u0 = ufl.sin(np.pi * x[0]) * ufl.sin(np.pi * x[1]) + ufl.sin(2*np.pi * x[0]
            ) * ufl.sin(4*np.pi * x[1])

# right hand side
f1 = fem.Function(V)
rhs1 = ufl.sin(np.pi * x[0]) * ufl.sin(np.pi * x[1])
f1_ = fem.Expression(rhs1, V.element.interpolation_points())
f1.interpolate(f1_)

rhs = [fem.Constant(msh, 0.0), f1]

# take the right hand side that was chosen before
f = rhs[example]

f = fem.Constant(msh, 0.0)

In [7]:
# create a list of solutions
# to add u0, we need to transform it into a dolfin function at first
# set u_n = u_0 (interpolate u0 on the grid to obtain the correct data type)

u_n = fem.Function(V)
u0_ = fem.Expression(u0, V.element.interpolation_points())
u_n.interpolate(u0_)
lst_solutions = [u_n]

In [8]:
# diffusivity coefficient \kappa

kappa = 0.1

# number of time steps

num_steps = 200


In the following block, the FEM solution is computed.

In [9]:
start = time.perf_counter()

for i in range(num_steps):

    ### ================================================================
    ### Construct the variational problem
    ### Discretize the time by finite differences to obtain the equation
    ### u^{n+1} = h ( f^{n+1} + \kappa \Delta u^{n+1} ) + u^{n}
    ### where u^{n}, f^{n} are the respective functions at time step n
    ### in the following, u^{n} and f^{n} are simply denoted by u and f,
    ### respectively.
    ### then derive the weak formulation


    # initialize trial and test function

    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)  

    ### =================================================================
    ### bilinear form and linear form
    ### a(u, v) = \int_{\Omega} uv + h \kappa \nabla u \cdot \nabla v dx
    ### l(v) = \int_{\Omega} (hf + u^{n})v dx

    # inner denotes the standard inner product, for real-valued functions
    # this is a normal product     

    a = (inner(u, v)) * dx + h * inner(kappa * grad(u), grad(v)) * dx
    l = inner(h * f + u_n, v) * dx

    ### ===============================================================
    ### define the linear problem
    ### a(u, v) = l(v) for all v in H^1_0 ( \Omega )

    # since the system matrix of the linear problem is symmetric and positive
    # definite, we use the CG method as solver
    # Algebraic Multigrid (AMG) as precoditioner
    problem = LinearProblem(a, l, bcs=[bc], petsc_options={
        "ksp_type": "cg",
        "ksp_rtol": 1e-8,
        "ksp_max_it": 2000,
        "pc_type": "hypre",
        "pc_hypre_type": "boomeramg",})
    uh = problem.solve()
    
    # set initial condition of next time step to current solution

    u_n.x.array[:] = uh.x.array

    # add solution at current time step to solutions 

    lst_solutions.append(u_n.copy())

end = time.perf_counter()
# run time
print("runtime: ", end - start)

runtime:  38.50160935200984


Compare the FEM solution with the analytic solution of the IBVP. If $f \equiv 0$, it is given by
$$
u(t, x, y) = \exp(- 2 \kappa \pi^2 t) \sin( \pi x) \sin ( \pi y ) + \exp(- 20 k \pi^2 t) \sin(2 \pi x) \sin (4 \pi y).
$$
If $f(x, y) = \sin (\pi x) \sin (\pi y)$, then
\begin{align*}
u(t, x, y) &= \left( 1 - \frac{1}{2 \kappa \pi^2} \right) \sin ( \pi x ) \sin ( \pi y ) \mathrm{e}^{- 2 \kappa \pi^2 t} + \sin ( 2 \pi x ) \sin ( 4 \pi y ) \mathrm{e}^{-20 \kappa \pi^2 t} \\ 
        & \qquad + \frac{1}{2 \kappa \pi^2} \sin ( \pi x ) \sin ( \pi y ).
\end{align*}

In [10]:
### ============================================================================================
### analytic solution
### create a normal python function since this easier to plot

# create a ufl expression for the exact solution at the i-th time step

def u_exact_ufl(i):
    
    if example == 0:    
        # t = i * h
        u_e = ufl.exp(- 2 * kappa * np.pi**2 * i * h) * ufl.sin(np.pi * x[0]) * ufl.sin(
            np.pi * x[1]) + ufl.exp(- 20 * kappa * np.pi**2 * i * h) * ufl.sin(2 * np.pi * x[0]
            ) * ufl.sin(4 * np.pi * x[1])
        return u_e

    elif example == 1:
        u_e = (1 - 1 / (2 * kappa * np.pi**2)) * ufl.sin(np.pi * x[0]) * ufl.sin(
            np.pi * x[1]) * ufl.exp(- 2 * kappa * np.pi**2 * i * h) + ufl.sin(2 * np.pi * x[0]
            ) * ufl.sin(4 * np.pi * x[1]) * ufl.exp(- 20 * kappa * np.pi**2 * i * h) + 1 / (
            2 * kappa * np.pi**2) * ufl.sin(np.pi * x[0]) * ufl.sin(np.pi * x[1])
        return u_e

We measure the error in the $L^2$ norm, defined by
$$
\| v \|_{L^2} := \left( \int_0^T \int_{(0,1)^2} |v(t, \mathbf{x})|^2\ \mathrm{d}\mathbf{x}\ \mathrm{d}t \right)^{1/2}.
$$
for $v \in L^2([0, T] \times (0, 1)^2)$ abd the $L^{\infty}$ norm which is defined as
$$
\| v \|_{L^{\infty}} = \operatorname{ess sup}_{(t, \mathbf{x}) \in [0, T] \times (0, 1)^2} | v(t, \mathbf{x}) |.
$$
for $v \in L^{\infty} ([0, T] \times (0, 1)^2)$.

In [11]:
### ================================================================================
### Compute the L^2 and L^{\infty} error

# we have to construct a ufl expression again
# Function space for exact solution, should have a higher degree
V_exact = fem.functionspace(msh, ("Lagrange", 5))
L_2_errs = []
L_infty_errs = []

# leave out 0-th time step since solution is exact by definition
for i in range(1, len(lst_solutions)):
    u_exact = fem.Function(V_exact)
    u_e = u_exact_ufl(i)
    u_exact_expr = fem.Expression(u_e, V_exact.element.interpolation_points())
    u_exact.interpolate(u_exact_expr)

    # L^2 errors
    L_2_err = msh.comm.allreduce(
                fem.assemble_scalar(fem.form((lst_solutions[i] - u_exact) ** 2 * ufl.dx)), 
                op=MPI.SUM)
    L_2_errs.append(L_2_err)

    # L^{\infty} errors
    # interpolate the FEM solution on the larger grid where the analytic solution is defined
    u_fem = fem.Expression(lst_solutions[i], V_exact.element.interpolation_points())
    u_fem_sol = fem.Function(V_exact)
    u_fem_sol.interpolate(u_fem)
    L_infty_err = np.max(np.abs(u_exact.x.array - u_fem_sol.x.array))
    L_infty_errs.append(L_infty_err)

# L^2 norm
display(Math(r'L^2\ \text{error} = ' + str(np.sqrt(h * np.sum(L_2_errs)))))

# for the L^{\infty} error we only have to the take the maximum over the errors at all
# time steps
display(Math(r'L^\infty\ \text{error} = ' + str(np.max(L_infty_errs))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Animation

In [12]:
### ===============================================================================
### Create an animation using pyvista (https://docs.pyvista.org/).

# Extract dolfinx mesh

cells, cell_types, points = plot.vtk_mesh(V)

# Make the grid

grid = pv.UnstructuredGrid(cells, cell_types, points)

# Add initial scalars

values = u_n.x.array
grid.point_data["values"] = values
base_points = grid.points.copy()

# Plotter

plotter = pv.Plotter(notebook=False, off_screen=True)
actor = plotter.add_mesh(
    grid,
    scalars="values",
    lighting=False,
    show_edges=True,
    scalar_bar_args={"title": "Height"},
    clim=[np.min(lst_solutions[0].x.array), np.max(lst_solutions[0].x.array)],
)

# set camera position and zoom such that plot is well visible

plotter.camera.focal_point = (0, 0, 0.2)
plotter.camera.position = (3, 3, 2)
plotter.camera.zoom(0.9)
plotter.open_movie("2d_heat_equation.mp4")

# Keep original points array

pts = grid.points.copy()

# number of frames should be equal to number of time steps

for frame in range(num_steps):

    points = base_points.copy()
    Z = lst_solutions[frame].x.array

    # update coordinates

    pts[:, 2] = Z.ravel()           # modify Z and 
    grid.points = pts               # update mesh points

    # update scalars

    grid.point_data["values"] = Z
    plotter.write_frame()           # triggers render

plotter.close()



2025-12-10 11:43:24.516 ( 204.727s) [    7852729E1140]vtkXOpenGLRenderWindow.:1458  WARN| bad X server connection. DISPLAY=
